În acest Colab, vă prezentăm un ghid pas cu pas despre cum să adjustați Whisper pentru orice set de date multilingv folosind Hugging Face 🤗 Transformers. Aceasta este o versiune mai „practică” a [postării de blog](https://huggingface.co/blog/fine-tune-whisper).


## Pregătirea mediului


În primul rând, să încercăm să asigurăm un GPU decent pentru Colab nostru! Din păcate, devine mult mai greu să obții acces la un GPU bun cu versiunea gratuită a Google Colab. Cu toate acestea, cu Google Colab Pro, nu ar trebui să aveți probleme în a primi un GPU V100 sau P100. Pentru a obține un GPU, faceți clic pe _Runtime_ -> _Change runtime type_, apoi schimbați _Hardware accelerator_ de la _CPU_ la unul dintre GPU-urile disponibile, de ex. _T4_ (sau mai bine dacă aveți unul disponibil). Apoi, faceți clic pe „Connect T4” în colțul din dreapta sus al ecranului (sau „Connect {V100, A100}” dacă ați selectat un alt GPU).


Putem verifica că ni s-a atribuit un GPU și să vedem specificațiile acestuia:


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun May  5 09:15:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0              31W /  70W |   3179MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Vom folosi mai multe pachete populare Python pentru adjustarea fină modelul Whisper. Vom folosi `datasets[audio]` pentru a descărca și pregăti datele noastre de antrenament, alături `transformers` și `accelerate`  pentru a încărca și antrena modelul nostru Whisper. De asemenea, vom solicita librărie `soundfile` pentru a preprocesa fișierele audio, `evaluate`  și `jiwer` pentru a evalua performanța modelului nostru și `tensorboard` pentru a înregistra valorile noastre. În cele din urmă, vom folosi `gradio` pentru a construi un demonstrație strălucitoare a modelului nostru fin reglat.


In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

Vă sfătuim insistent să încărcați punctele de control al modelului direct în [Hugging Face Hub](https://huggingface.co/) în timpul antrenamentului. Hub-ul oferă:
 - Control integrat al versiunii: puteți fi sigur că niciun punct de control al modelului nu este pierdut în timpul antrenamentului.
 - Jurnalele Tensorboard: urmăriți valorile importante pe parcursul antrenamentului.

 - Carduri model: documentează ceea ce face un model și cazurile de utilizare ale acestuia.
 - Comunitate: o modalitate ușoară de a partaja și de a colabora cu comunitatea!
 Conectarea notebook-ului la Hub este simplă - necesită pur și simplu introducerea simbolului de autentificare Hub atunci când vi se solicită. Găsiți indicativul de autentificare Hub [aici](https://huggingface.co/settings/tokens):


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Încărcați setul de date


Folosind 🤗 Seturi de date, descărcarea și pregătirea datelor este extrem de simplă. Putem descărca și pregăti diviziunile într-o singură linie de cod. Am folosit date colectate cu ajutorul [colectorului de date](https://github.com/Yehoward/iazar-datacollector) în cadrul proiectului Tekwill Junior Ambasadors. **Datele sunt private, ca să nu fie utilizate contra donatorilor noștri!**


In [ ]:
from datasets import load_dataset, DatasetDict

dataset = DatasetDict()


pretrained_model = {
    "pretrained_model_name_or_path": "Yehoward/whisper-small-ro",
    "revision": "b3853106ca127f6daa19cd11058138398e7af174"
}

dataset_name = "Yehoward/iazar-date"
dataset["train"] = load_dataset(dataset_name, split="train", use_auth_token=True)
dataset["test"] = load_dataset(dataset_name, split="test", use_auth_token=True)

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 37
    })
    test: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 15
    })
})


## Pregătim Extractor de caracteristici, Tokenizer și Date


Conducta TAV(Transcrierea Automată a Vocii - eng. [ASR]) poate fi descompusă în trei etape:
 1. Un extractor de caracteristici care preprocesează intrările audio brute
 2. Modelul care realizează maparea secvență-la-secvență
 3. Un tokenizer care post-procesează ieșirile modelului în format text. În 🤗 Transformers, modelul Whisper are asociat un extractor de caracteristici și un tokenizer, numit [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor) și [WhisperTokenizer]( respectiv https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer). Vom parcurge detaliile pentru configurarea extractorului de caracteristici și tokenizerului unul câte unul!


### Încărcați WhisperFeatureExtractor


Extractorul caracteristicii Whisper efectuează două operațiuni:
 1. Tampează / trunchiază intrările audio la 30 de secunde: orice intrări audio mai scurte de 30 de secunde sunt completate la 30 de secunde cu tăcere (zero), iar cele mai lungi de 30 de secunde sunt trunchiate la 30 de secunde
 2. Convertește intrările audio la caracteristicile de intrare _log-Mel spectrogram_, o reprezentare vizuală a sunetului și forma intrării așteptate de modelul Whisper


<figura> <img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%"> <figcaption align = "center „><b>Figura 2:</b> Conversia matricei audio eșantionate în spectrograma log-Mel. Stânga: semnal audio unidimensional eșantionat. Dreapta: spectrograma log-Mel corespunzătoare. Sursa figurii: <a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>. </figcaption>


Vom încărca extractorul de caracteristici de la punctul de control pre-antrenat cu valorile implicite:


In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

### Încărcăm WhisperTokenizer


Modelul Whisper produce o secvență de _ID-uri token_. Tokenizer-ul corelează fiecare dintre aceste ID-uri de simbol cu șirul de text corespunzător. Pentru română, putem încărca tokenizatorul pre-antrenat și îl putem folosi pentru adjustare fină fără alte modificări. Trebuie doar să specificăm limba țintă și sarcina. Aceste argumente informează tokenizerul să prefixeze limbajul și simbolurile de activitate la începutul secvențelor de etichete codificate:


In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(language="Romanian", task="transcribe", **pretrained_model)

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

### Combinăm pentru a crea un WhisperProcessor


Pentru a simplifica utilizarea extractorului de caracteristici și tokenizerului, le putem _încheia_ pe ambele într-o singură clasă `WhisperProcessor`. Acest obiect procesor moștenește de la `WhisperFeatureExtractor` și `WhisperProcessor` și poate fi utilizat pe intrările audio și predicțiile modelului după cum este necesar. Făcând acest lucru, trebuie să urmărim doar două obiecte în timpul antrenamentului: „procesorul” și „modelul”:


In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(language="Romanian", task="transcribe", **pretrained_model)

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

### Pregătim datele


Să tipărim primul exemplu al setului de date pentru a vedea în ce formă sunt datele:


In [ ]:
print(dataset["train"][0])

{'audio': {'path': '2024-04-23T20:12:56.wav', 'array': array([-1.52587891e-05, -1.52587891e-05, -1.52587891e-05, ...,
        2.76184082e-03,  1.49536133e-03,  1.92260742e-03]), 'sampling_rate': 16000}, 'transcription': 'Îți recomand să bei niște vin pentru stomac.'}


Deoarece audiouril pot să fie eșantionate în 48 kHz, trebuie să le convertim în 16 kHz înainte de a-l trece la extractorul caracteristicii Whisper, 16kHz fiind rata de eșantionare așteptată de modelul Whisper. Vom seta intrările audio la rata de eșantionare corectă folosind metoda [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column) a setului de date. Această operațiune nu modifică sunetul în loc, ci mai degrabă semnalizează către „seturi de date” pentru a reeșantiona probe audio _din zbor_ prima dată când sunt încărcate:


In [ ]:
from datasets import Audio

dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

Reîncărcarea primului eșantion audio din setul de date îl va reeșantiona la rata de eșantionare dorită:


Acum putem scrie o funcție pentru a pregăti datele noastre pentru model:
 1. Încărcăm și reeșantionăm datele audio apelând `batch["audio"]`. După cum s-a explicat mai sus, 🤗 Datasets efectuează orice operațiuni de reeșantionare necesare din mers.
 2. Folosim extractorul de caracteristici pentru a calcula caracteristicile de intrare ale spectrogramei log-Mel din matricea noastră audio unidimensională.
 3. Codificăm transcripțiile pentru a eticheta ID-uri prin utilizarea tokenizer-ului.


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcription"]).input_ids
    return batch

Putem aplica funcția de pregătire a datelor la toate exemplele noastre de antrenament folosind metoda `.map` a setului de date. Argumentul `num_proc` specifică câte nuclee CPU să folosească. Setarea `num_proc` > 1 va activa multiprocesarea. Dacă metoda `.map` se blochează cu multiprocesare, setați `num_proc=1` și procesați setul de date secvenţial.


In [ ]:
dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/37 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

## Instruire și evaluare


Acum că ne-am pregătit datele, suntem gata să ne aprofundăm în conducta de antrenament. [🤗 Antrenorul](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) va face o mare parte din munca grea pentru noi. Tot ce trebuie să facem este:
 - Încărcăm un punct de control pre-antrenat: trebuie să încărcăm un punct de control pre-antrenat și să-l configuram corect pentru antrenament.
 - Definiți un colator de date: colatorul de date preia datele noastre preprocesate și pregătește tensorii PyTorch gata pentru model.
 - Valori de evaluare: în timpul evaluării, dorim să evaluăm modelul utilizând valoarea [rata de eroare a cuvintelor (WER)](https://huggingface.co/metrics/wer). Trebuie să definim o funcție `compute_metrics` care se ocupă de acest calcul.
 - Definiți configurația antrenamentului: aceasta va fi folosită de către 🤗 Trainer pentru a defini programul de antrenament. Odată ce am ajustat modelul, îl vom evalua pe datele testului pentru a verifica dacă l-am antrenat corect pentru a transcrie vorbirea în limba română.


### Încărcăm un punct de control pre-antrenat


Vom începe cursa noastră de adjustare fină de la punctul de control pre-antrenat Whisper `small`, Greutățile căruia le vom încărca de pe Hugging Face Hub. Din nou, acest lucru este banal prin utilizarea 🤗 Transformers!


In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(**pretrained_model)

Putem dezactiva sarcina de detectare automată a limbii efectuată în timpul inferenței și să forțăm generarea modelului în limba română. Pentru a face acest lucru, setăm [langauge](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.language) și [sarcina](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.task) argumente ale config. De asemenea, vom seta orice [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids) la None, deoarece aceasta era modalitatea moștenită de a seta argumente privind limbajul și sarcina:


In [ ]:
model.generation_config.language = "romanian"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Definim un colector de date


Colaboratorul de date pentru un model de vorbire secvență-la-secvență este unic în sensul că tratează „input_features” și „labels” independent: „input_features” trebuie să fie gestionat de extractor de caracteristici, iar „labels” de către tokenizer. `Input_features` sunt deja completate la 30 de secunde și convertite într-o spectrogramă log-Mel de dimensiune fixă prin acțiunea extractorului de caracteristici, așa că tot ce trebuie să facem este să convertim `input_features` în tensori PyTorch în loturi. Facem acest lucru folosind metoda `.pad` a extractorului de caracteristici cu `return_tensors=pt`. Pe de altă parte, „etichetele” nu sunt căptușite. Mai întâi adăugăm secvențele la lungimea maximă din lot folosind metoda `.pad` a tokenizerului. Jetoanele de umplutură sunt apoi înlocuite cu `-100`, astfel încât aceste jetoane **n-o** să fie luate în considerare la calcularea pierderii. Apoi tăiem simbolul BOS de la începutul secvenței de etichete, pe măsură ce îl anexăm mai târziu în timpul antrenamentului. Putem folosi `WhisperProcessor` pe care l-am definit mai devreme pentru a efectua atât extractorul de caracteristici, cât și operațiunile de tokenizer:


In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Să inițializam colatorul de date pe care tocmai l-am definit:


In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Valori de evaluare


Vom folosi indicatorul pentru rata de eroare a cuvântului (WER), indicatorul „de facto” pentru evaluarea sistemelor ASR. Pentru mai multe informații, consultați [WER docs](https://huggingface.co/metrics/wer). Vom încărca valoarea WER din 🤗 Evaluate:


In [ ]:
import evaluate

metric = evaluate.load("wer")

Apoi trebuie pur și simplu să definim o funcție care preia predicțiile modelului nostru și returnează metrica WER. Această funcție, numită `compute_metrics`, înlocuiește mai întâi `-100` cu `pad_token_id` în `label_ids` (anulând pasul pe care l-am aplicat în colatorul de date pentru a ignora corect token-urile padded în pierdere). Apoi decodifică ID-urile prezise și etichetează în șiruri. În cele din urmă, calculează WER între predicții și etichetele de referință:


In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Definim configurația de antrenament


În pasul final, definim toți parametrii legați de antrenament. Pentru mai multe detalii despre argumentele antrenamentului, consultați [Seq2SeqTrainingArguments docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-ro",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=200,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=200,
    eval_steps=200,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

**Notă**: dacă nu doriți să încărcați punctele de control al modelului în Hub, setați `push_to_hub=False`.


Putem transmite argumentele antrenamentului către 🤗 Trainer împreună cu modelul nostru, setul de date, colatorul de date și funcția `compute_metrics`:


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

Vom salva o dată obiectul procesorului înainte de a începe antrenamentul. Deoarece procesorul nu poate fi antrenat, nu se va schimba pe parcursul antrenamentului:


In [ ]:
processor.save_pretrained(training_args.output_dir)

[]

### Instruire


Formarea a durat 8-10 min, în funcție de GPU-ul dvs. sau de cel alocat acestui Google Colab. Dacă utilizați acest Google Colab direct pentru a regla fin un model Whisper, ar trebui să vă asigurați că antrenamentul nu este întrerupt din cauza inactivității. O soluție simplă pentru a preveni acest lucru este să inserați următorul cod în consola acestei file (_clic dreapta al mouse-ului_ -> _inspect_ -> _fila Consolă_ -> _inserați codul_).


```funcția javascript ConnectButton(){ console.log("Conexiune împinsă"); document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() } setInterval(ConnectButton, 60000); ```


Memoria GPU de vârf pentru configurația de antrenament dată este de aproximativ 15,8 GB. În funcție de GPU-ul alocat Google Colab, este posibil să întâmpinați o eroare CUDA `"out-of-memory"` când lansați antrenamentul. În acest caz, puteți reduce `per_device_train_batch_size` treptat cu factori de 2 și utilizați [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps) pentru a compensa. Pentru a lansa antrenamentul, executați pur și simplu:


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
200,0.000500,0.820713,46.265060


TrainOutput(global_step=200, training_loss=0.09695319784339518, metrics={'train_runtime': 738.5774, 'train_samples_per_second': 4.333, 'train_steps_per_second': 0.271, 'total_flos': 7.1396028039168e+17, 'train_loss': 0.09695319784339518, 'epoch': 66.66666666666667})

Scrim argumentele pentru salvarea modeluiui pe 🤗

In [ ]:
kwargs = {
    "dataset_tags": dataset_name,
    "dataset": "Date audio colectate în cadrul proiectului TekWill",  # nume pentru setul de date
    "dataset_args": "split: test",
    "language": "ro",
    "model_name": "Whisper Small Ro - Iazar",  # nume pentru modelul nostru
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

Rezultatele antrenamentului pot fi acum încărcate în Hub. Pentru a face acest lucru, executați comanda `push_to_hub` și salvați obiectul preprocesor pe care l-am creat:


In [ ]:
trainer.push_to_hub(**kwargs)

CommitInfo(commit_url='https://huggingface.co/Yehoward/whisper-small-ro/commit/76b10b323b8d258a424df10c1464a4dc1f4c372e', commit_message='End of training', commit_description='', oid='76b10b323b8d258a424df10c1464a4dc1f4c372e', pr_url=None, pr_revision=None, pr_num=None)